# About This Page
The __goal of this page__ is to provide users the ability to quickly see 
_"a summary of what happened on the dome during the previous night"_. See [MORE About this page](#More-about-this-page)
## What has been added recently?
- a little face lift.  Data sources. Internal cross linking.
- See also: [Previously added](#Previously-Added)

In [ ]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, v, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
# day_obs = '2024-09-25' # 2024-12-06 2024-11-20 Value to use for local testing (Summit)
# 2024-11-30
# ComCam on-sky campaign: 24 October to 11 December 2024
# 2024-11-30: Exposures(372,2013) time: 9:18:47 + 1:49:23
# 2024-12-01: Exposures(510,1401) time: 5:42:08 + 2:32:42, has Good      <<<
# 2024-12-02: Exposures(738) time: 3:02:05, + LSSTComCam(423)
# 2024-12-07: Exposures(612) time: 3:13:06
# 2024-12-08: Exposures(738) time: 2:40:54
# 2024-12-10: Exposures(614) time: 2:45:17, has Good
# 2024-12-11: Exposures(612) time: 2:15:08
day_obs = "2024-12-02"  # TODO Change to 'YESTERDAY' and 'TODAY' to test with default before push
# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push
period = "2h"  # TODO change to 4h before push
verbose = "false"  # TODO change to false before push, else true
warning = "false"  # TODO change to false before push, else true

In [ ]:
# IMPORT everything
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    DEV_ENV = False
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        "TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP."
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    DEV_ENV = True
    dev_msg = "Imported lsst.ts.logging_and_reporting from local packages."
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
import lsst.ts.logging_and_reporting.time_logs as tl
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink, mdlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf
)  # TODO try with "usdf" before push (also "summit")

In [ ]:
# Validate parameters, return usable ones
usable, error = ut.fallback_parameters(
    day_obs, number_of_days, period, verbose, warning
)
if error:
    print(error)

date = ut.get_datetime_from_dayobs_str(usable["day_obs"])
# date:  is EXCLUSIVE (upto, but not including)
days = usable["number_of_days"]
period = usable["period"]
verbose = usable["verbose"]
warning = usable["warning"]

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [ ]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,
    warning=warning,
    limit=5000,
    exclude_instruments=[],  # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

In [ ]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
tlist = list()
tlist.append(f"## Showing data for {min_date.date()} to {max_date.date()}")
tlist.append(f"- **Instruments:** {instrum_str}")
tlist.append(f"- **Server:** {server}")
tlist.append(
    f'- Most recently run on: **{dt.datetime.now().replace(microsecond=0).isoformat(sep=" ")}**'
)
md("\n".join(tlist))

# Table of Contents
* [About This Page](#About-This-Page)
* [Almanac (BETA)](#Almanac-BETA)
* [Night Report (BETA)](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Links to related resources (BETA)](#Links-to-related-resources-BETA)
* [Time Accounting (DRAFT)](#Time-Accounting-BETA)
* [Jira Tickets (BETA)](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [Data Log (BETA)](#Data-Log-BETA)
* [Narrative Log (BETA)](#Narrative-Log-BETA)
* [Developer Only Section (REMOVE)](#Developer-Only-Section-REMOVE)
* [More about this page](#More-about-this-page)

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [ ]:
display(allsrc.alm_src.dataframe)
md(f"Moon Illumination: {allsrc.alm_src.moon_illum:.0%}")

#### Sources Used

In [ ]:
for name, url in allsrc.alm_src.sources.items():
    md(f"1. [{name}]({url})")

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [ ]:
commissioning_plan = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testCycle/BLOCK-R164"
md(
    f"""
Also see: {mdlink(commissioning_plan, "2024-12-09 Commissioning Plan")}
*(we need a general way to refer to plan for the a specific night)*
"""
)

In [ ]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

#### Sources Used

In [ ]:
for name, url in allsrc.nig_src.sources.items():
    md(f"1. [{name}]({url})")

[BACK to the TOC](#Table-of-Contents)

## Links to related resources <font style="background-color:green; color:white; font-size:20px">BETA</font>
These tools were created outside of the TSSW Logging and Reporting group.  For more information about any of these, please see the respective tool owner.

In [ ]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)

nightsum_link_url = f"{server}/times-square/github/lsst/schedview_notebooks/nightly/scheduler-nightsum?day_obs={min_date.date()}&visit_origin=lsstcomcam"
nightsum_link_string = f"link to nightsum for day_obs = {min_date.date()}"

scriptqueue_url = f"https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/schedview_notebooks/nightly/EvaluateScriptQueue?day_obs_min={min_date.date()}&day_obs_max={max_date.date()}&time_order=newest+first&show_salIndex=all&ts_hide_code=1"

md(
    f"""
#### DDV part of RubinTV
  {mdlink(DDV, 'RubinTV Visualization')}

#### Chronograf links
{mdlink(f'{server}/chronograf','Chronograph main page on {server}')}

#### Scheduler Analysis Nightly Summary
{mdlink(nightsum_link_url, nightsum_link_string)}

### Integrated Narrative Log + EFD
{mdlink(scriptqueue_url, f'ScriptQueue Miner for {min_date.date()}')}

"""
)

## Time Accounting <font style="background-color:green; color:white; font-size:20px">BETA</font>
TODO:
- Time-loss for Fault and Weather  (very rarely in narrativelog/messages)

In [ ]:
# Exposure Tally (v2)
df, footnotes = allsrc.time_account()
display(df)
# display(footnotes)

In [ ]:
mstr = ""
for num, fn in footnotes.items():
    mstr += f"- ({num}) {fn}\n"
md(mstr)

In [ ]:
# Exposure Tally
# tally = allsrc.night_tally_observation_gaps()
# if tally:
#    display(pd.DataFrame(tally))

In [ ]:
# - (1) There is no practical way to get detector read-out time.  A value of 2.41 seconds per exposure is used.
# - (2) There is currently no simple way to get slew times.  We SlewTime to find its way into the Consolidated Database #eventually.

#### Sources Used
- Almanac
- Exposure Log
- Narraitve Log

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
#### <font style="background-color:#ffff99">Possible Changes</font>
- [ ] Observing Operations - OBS
- [ ] TODO: better tickets using Jira API to get selected fields (which) from tickets

In [ ]:
if allsrc.urls:
    text = ""
    for url in allsrc.urls:
        text += f"- {mdpathlink(url)}\n"
    md(text)

In [ ]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()

if tickets:
    mdstr = "#### Nightly Jira BLOCKs -- Commisioning Plan"
    for day, url_list in tickets.items():
        mdstr += f"\n - {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - {mdlink(ticket_url, ticket_url.replace(front,""))}'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(
        f"No tickets found using: {mdlink(endpoint_url,'API Data')} in `confluence_url`."
    )

#### Sources Used
- Almanac
- Exposure Log
- Narraitve Log

[BACK to the TOC](#Table-of-Contents)

## Data Log <font style="background-color:green; color:white; font-size:20px">BETA</font>
Count the number of exposures (tally) for each instrument. For instruments that have exposures, partition the tally by various attributes and the exposure quality flags.

In [ ]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f"#### {instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures")
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name, df in df_dict.items():
        if df.empty:
            continue
        #! md("##### " + field_name.title().replace("_", " "))
        display(HTML(df.style.hide().to_html(escape=False)))

#### Sources Used

In [ ]:
for name, url in allsrc.exp_src.sources.items():
    md(f"1. [{name}]({url})")

[BACK to the TOC](#Table-of-Contents)

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 
#### <font style="background-color:#ffff99">Possible Changes</font>
- [ ] Group messages by hour? Might be great when there many simple messages minutes apart. Less so when tracebacks are included.
- [ ] Truncate text messages to N chars (ending with "..." as link if truncated, link goes to page with full message record including text and all attributes)
- [ ] Detect some kinds of "blocks" and render in smaller text (+ other format changes)

In [ ]:
allsrc.nar_src

In [ ]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

#### Sources Used

In [ ]:
for name, url in allsrc.nar_src.sources.items():
    md(f"1. [{name}]({url})")

[BACK to the TOC](#Table-of-Contents)

# More about this page
__Intended users include:__ management, commissioning scientists , observers, and test scientists. Its likely that the __role of each user will determine which sections__ in this page are most important to them.  __Discussion__ about this and similar pages can be found on the slack *#ts-logging* channel. 

[BACK to the Top](#About-This-Page)

## Previously Added
- Stakeholders decided that supported instruments will be: LSSTComCam, LSSTCam, and LATISS. Of those, LSSTCam is not supported in ConsDB and limitations in LATISS will force some of the requested fields for ExposureDetail to be given as NA.
- Various minor, mostly cosmetic changes to: Night Report, Almanac, Time Accounting, Data Log, ExposureDetail (drill down page from Data Log)
- Removed Consolidated Database section. Most of it is in ExposureDetail linked to from Data Log.
- Added optional WARNING behavior to alert when no records are found, instruments are excluded from results, etc. See "warning" parameter in left sidebar to enable/disable it.
- <font style="background-color:#ffff99">Embedded comments and questions to reader (Yellow background)</font>
- <font style="background-color:red; color:#90EE90">Errors from APIs are displayed in this page (Red background).</font>
- Try dates: 2024-12-01 (ComCam on-sky), 2024-09-25,

[BACK to the Top](#About-This-Page)

-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.
This may also contain sections that have moved out of the user section.

## Overview 


In [ ]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
"""
)

## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [ ]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(index=False, render_links=True, escape=False)
display(HTML(table))

## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Available Consolidated Database fields
It should be easy to add data to this report when it corresponds to any of the fields listed below. 

**Format:** *instrument/tablename/fieldname*

TODO: index by fieldname, multi-index columm = (Instrument, TableName), val = yes or NA

In [ ]:
verbose = True
if verbose:
    #! txt = '\n- '.join(allsrc.cdb_src.all_available_fields)
    #! md(f'- {txt}')
    table = ""
    table += "\n| instrument | tablename | fieldname |"
    table += "\n| :--------: | :-------: | :-------: |"
    for field in allsrc.cdb_src.all_available_fields:
        table += f'\n| {field.replace("/"," | ")} |'
    md(table)

## Section overviews moved here

In [ ]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

In [ ]:
# Time Log
#! allrep.exp_rep.time_log_as_markdown()

## Finale

In [ ]:
print(f"Elapsed time (excluding code import): {timer.toc:.1f} seconds")
print(f"Finished {str(dt.datetime.now().replace(microsecond=0))} UTC")

In [ ]:
allsrc